**EVENT DETECTION**


The script provides a comprehensive implementation for eye movement classification using both the I-VT (Identification by Velocity Threshold) and I-DT (Identification by Dispersion Threshold) algorithms, as described in the paper.

---
The script analyzes eye-tracking data to detect and classify eye movements into fixations and saccades. It employs two distinct methods:

*   I-VT (Identification by Velocity Threshold): This method classifies eye movements based on the velocity between consecutive gaze points. If the velocity is below a predefined threshold, the movement is classified as a fixation; otherwise, it is classified as a saccade.

*   I-DT (Identification by Dispersion Threshold): This method uses the dispersion of gaze points within a time window to classify movements. If the points remain within a certain spatial threshold, it indicates a fixation.







In [ ]:
#Installing required libraries
!pip install pandas

In [ ]:
#Import required libraries
import pandas as pd
import numpy as np

Section 1: Data Loading
-----------------------
This section reads a CSV file into a pandas DataFrame. This file contains eye-tracking data including timestamps, gaze points, participant information, validity of the data points and other features.


In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv('ExampleData.csv')

Section 2: Data Filtering
-------------------------
* Filters the DataFrame to only include data for 'Participant1'. This is useful in scenarios where the dataset contains multiple participants, and analysis needs to be participant-specific.

* Further filters the data to include only rows where both the 'Validity left' and 'Validity right' fields are 'Valid'.

* This step ensures that only high-quality, valid gaze data is considered in the analysis.

In [ ]:
# Filter the DataFrame for rows where 'Participant name' is 'Participant1'
df = df[df['Participant name'] == 'Participant1']

# Further filter the DataFrame for rows where both 'Validity left' and 'Validity right' are 'Valid'
df = df[(df['Validity left'] == 'Valid') & (df['Validity right'] == 'Valid')]
df = df.reset_index(drop=True)



Section 3: Function Definitions - Identification by Velocity Threshold (I-VT)
-------------------------------
* This section contains definitions of several functions used for processing eye-tracking data.

In [ ]:
# Function: calculate_pixel_to_pixel_velocities
# Purpose: To compute velocities between consecutive gaze points. Output of this
# function is used to determine the type of eye movement (saccade or fixation)
# based on these velocities.
def calculate_pixel_to_pixel_velocities(timestamps, gaze_points):
    velocities = []
    for i in range(1, len(gaze_points)):
        delta_x = gaze_points[i][0] - gaze_points[i-1][0]
        delta_y = gaze_points[i][1] - gaze_points[i-1][1]
        delta_t = (timestamps[i] - timestamps[i-1]) / 1000.0  # Assuming timestamps are in milliseconds
        distance = np.sqrt(delta_x**2 + delta_y**2)
        velocity = distance / delta_t if delta_t > 0 else 0
        velocities.append(velocity)
    return velocities



# Function: classify_points
# Purpose: To classify each gaze point based on velocity. Points are classified as 'fixation' if the velocity
# is below a certain threshold, otherwise as 'saccade'. It initializes with the assumption that the first point is a fixation.
def classify_points(velocities, V_threshold):
    types = ['saccade']  # Pre-set the first point as saccade
    eye_movement_ids = [0]  # Start IDs from 1, assuming the first point is a saccade
    current_id = 1  # Starting ID
    last_type = 'saccade'  # Initialize the last type as saccade
    for velocity in velocities:
        if velocity < V_threshold:
            if last_type == 'fixation':
                types.append('fixation')
                eye_movement_ids.append(current_id)  # Continue the current fixation ID
            else:
                current_id += 1  # Start a new fixation
                types.append('fixation')
                eye_movement_ids.append(current_id)
                last_type = 'fixation'
        else:
            if last_type == 'saccade':
                types.append('saccade')
                eye_movement_ids.append(current_id)  # Continue the current saccade ID
            else:
                current_id += 1
                types.append('saccade')
                eye_movement_ids.append(current_id)
                last_type = 'saccade'
    return types, eye_movement_ids


# Function: calculate_fixation_centroids
# Purpose: To calculate the centroids of fixation points grouped by 'Eye Movement ID'.
# It filters for fixation points, groups them, calculates the mean (centroid) for each group, and returns these centroids.
def calculate_fixation_centroids(df):
    # Filter to get only the fixation points
    fixation_df = df[df['Eye movement type - IVT'] == 'fixation']
    # Group by 'Eye Movement ID' and calculate the mean for each group
    centroids = fixation_df.groupby('Eye Movement ID')[['Gaze point X', 'Gaze point Y']].mean().reset_index()
    # Optionally, rename the columns to indicate these are centroid coordinates
    centroids.rename(columns={'Gaze point X': 'Fixation Centroid X', 'Gaze point Y': 'Fixation Centroid Y'}, inplace=True)
    return centroids

Section 4: Data Processing - Eye Movement Event Detection
---------------------------
This section interpolates missing values in gaze points, applies the I-VT algorithm to classify eye movements, and calculates the centroids of each fixation.

In [ ]:
# Interpolates missing values for accurate velocity calculation.
df[['Gaze point X', 'Gaze point Y']] = df[['Gaze point X', 'Gaze point Y']].interpolate()

# Extracting timestamps and gaze points.
timestamps = df['Computer timestamp'].tolist()
gaze_points = list(zip(df['Gaze point X'], df['Gaze point Y']))

# Calculate velocities between gaze points and classify each point.
velocities = calculate_pixel_to_pixel_velocities(timestamps, gaze_points)
V_threshold = 1  # Threshold for classifying movements
types, eye_movement_ids = classify_points(velocities, V_threshold)

# Update the DataFrame with new columns for eye movement types and IDs.
df['Eye movement type - IVT'] = types
df['Eye Movement ID'] = eye_movement_ids

# Calculate fixation centroids and merge this data back into the main DataFrame.
centroids = calculate_fixation_centroids(df)
df = df.merge(centroids, on='Eye Movement ID', how='left')

# Prints the few rows of the updated DataFrame to verify the results.
print(df.iloc[390:420])

     Unnamed: 0  Computer timestamp  Project name Participant name  \
390         487          3090207226  EyeTracking1     Participant1   
391         488          3091023903  EyeTracking1     Participant1   
392         489          3091040569  EyeTracking1     Participant1   
393         490          3091057236  EyeTracking1     Participant1   
394         491          3091690578  EyeTracking1     Participant1   
395         492          3091840580  EyeTracking1     Participant1   
396         493          3091857247  EyeTracking1     Participant1   
397         494          3091873914  EyeTracking1     Participant1   
398         495          3092440588  EyeTracking1     Participant1   
399         496          3092457255  EyeTracking1     Participant1   
400         497          3093290599  EyeTracking1     Participant1   
401         498          3093307266  EyeTracking1     Participant1   
402         499          3093773938  EyeTracking1     Participant1   
403         500     

As a result of this section, the main DataFrame now includes fixation points calculated using the IVT algorithm. This dataframe is extracted from Tobii Pro Lab, it also has its classifications of fixations and saccades. You can compare between the implemented fixation and saccade classification and the classifications provided by Tobii Pro Lab. Note: The results may differ from those produced by Tobii Pro Lab due to variations in threshold settings and other filtering techniques. To align the outputs more closely, consider implementing additional filtering methods according to your specific needs. For example, you could exclude fixations that are too short in duration.